## Intent Classification

In [1]:
import pandas as pd
import numpy as np
from mxnet.gluon import nn, rnn
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import time
import itertools
from tqdm import tqdm
import multiprocessing as mp

In [2]:
train_raw = pd.read_csv("data/trainset.txt",names=['intent', 'entity', 'sentence'], sep='\t')
#validation_raw = pd.read_csv("data/test_hidden.txt",names=['intent', 'entity', 'sentence'], sep='\t')
validation_raw = pd.read_csv("data/validation.txt",names=['intent', 'entity', 'sentence'], sep='\t')

In [3]:
train_raw.head(30)

,intent,entity,sentence
0,area,EECCCCCCCCCCCCCCCCCCC,자강의 면적은 얼마 정도되는지 알려줄래
1,birth_date,CCCCCCCCCCCCEEECCCCCCCCCCCC,WIKI PEDIA로 변재일 생년월일을 알고 싶어
2,age,EEEEEEEEEEECCCCCCCCCCCCCCCCC,남쪽 물고기자리 알파 나이가 위키백과사전으로 얼마야
3,length,EEEECCCCCCCCCCCCCCCCCC,삼양터널의 총 길이 위키백과사전에서 뭐야
4,birth_place,EEEEEECCCCCCCCCCC,코니 윌리스의 태어난 곳은 뭐지
5,weight,CCCCCCCCCCCCEEEECCCCCCCCCCCCC,WIKI백과사전 검색 AA12의 무게가 얼만지 찾아봐
6,definition,CCCCCCCCCCCCCEEECCCCCCCC,WIKIPEDIA백과로 라이프 찾아서 말해줘
7,height,EEEEEEEECCCCCCCCCCCCCCCCCCC,송파 헬리오시티 구조물 높이 위키 피디아에서 뭐야
8,birth_date,CCCEEEEEECCCCCCCCCCCCCCC,검색 HLKVAM 언제 출생했는지를 검색해라
9,height,CCCCCCCCEEEEEECCCCCCCC,위키 피디아에 푸조 508 전고가 몇이야


### Intent Classification

#### 데이터 전처리

In [4]:
train_dataset = [(l, d) for d,l in zip(train_raw['intent'], train_raw['sentence'])]
valid_dataset = [(l, d) for d,l in zip(validation_raw['intent'], validation_raw['sentence'])]

In [5]:
seq_len = 32

length_clip = nlp.data.PadSequence(seq_len, pad_val="<pad>")

def preprocess(data):
    sent, entity = data
    char_sent = list(str(sent))
    char_entity = str(entity)
    return(length_clip(char_sent), len(sent),char_entity)

def preprocess_dataset(dataset):
    start = time.time()
    with mp.Pool() as pool:
        dataset = gluon.data.SimpleDataset(pool.map(preprocess, dataset))
    end = time.time()
    print('Done! Tokenizing Time={:.2f}s, #Sentences={}'
          .format(end - start, len(dataset)))
    return dataset


In [6]:
train_preprocessed  = preprocess_dataset(train_dataset)
valid_preprocessed  = preprocess_dataset(valid_dataset)

Done! Tokenizing Time=0.14s, #Sentences=9000
Done! Tokenizing Time=0.14s, #Sentences=1000


In [7]:
counter_sent   = nlp.data.count_tokens(itertools.chain.from_iterable([c for c, _, _ in train_preprocessed]))
counter_intent = nlp.data.count_tokens([c for _,_, c in train_preprocessed])

In [8]:
counter_intent

Counter({'age': 900,
         'area': 900,
         'belong_to': 900,
         'birth_date': 900,
         'birth_place': 900,
         'definition': 900,
         'height': 900,
         'length': 900,
         'weight': 900,
         'width': 900})

In [9]:
vocab_sent = nlp.Vocab(counter_sent, bos_token=None, eos_token=None, min_freq=15)
vocab_intent = nlp.Vocab(counter_intent, bos_token=None, eos_token=None, unknown_token=None, padding_token=None)

In [10]:
vocab_sent.idx_to_token[:10], vocab_intent.idx_to_token[:10], 

(['<unk>', '<pad>', ' ', 'I', '이', '색', '검', '의', '지', '아'],
 ['age',
  'area',
  'belong_to',
  'birth_date',
  'birth_place',
  'definition',
  'height',
  'length',
  'weight',
  'width'])

In [11]:
train_preprocessed_encoded  = [(vocab_sent[sent], length ,vocab_intent[entity])  for sent, length ,entity in train_preprocessed ]
valid  = [(vocab_sent[sent], length ,vocab_intent[entity])  for sent, length ,entity in valid_preprocessed ]

In [12]:
train, test = nlp.data.train_valid_split(train_preprocessed_encoded, valid_ratio=0.1)

In [13]:
nbatch = 30
batchify_fn = nlp.data.batchify.Tuple(nlp.data.batchify.Stack(),
                                      nlp.data.batchify.Stack('float32'),
                                      nlp.data.batchify.Stack())

train_dataloader  = gluon.data.DataLoader(train, batch_size=nbatch, batchify_fn=batchify_fn, shuffle=True)
test_dataloader  = gluon.data.DataLoader(test, batch_size=nbatch, batchify_fn=batchify_fn, shuffle=True)
valid_dataloader  = gluon.data.DataLoader(valid, batch_size=nbatch, batchify_fn=batchify_fn, shuffle=True)

#### 모델링 

In [14]:
class IntentClassification(gluon.HybridBlock):
    def __init__(self, vocab_size, vocab_out_size, num_embed, seq_len, hidden_size, **kwargs):
        super(IntentClassification, self).__init__(**kwargs)
        self.seq_len = seq_len
        self.hidden_size = hidden_size 
        self.vocab_out_size = vocab_out_size
        with self.name_scope():
            self.embed = nn.Embedding(input_dim=vocab_size, output_dim=num_embed)
            self.bigru = rnn.GRU(self.hidden_size, dropout=0.2, bidirectional=True)
            self.dense_prev = nn.Dense(10, flatten=False)
            self.dense = nn.Dense(self.vocab_out_size)  
            
    def hybrid_forward(self, F ,inputs, length):
        em_out = self.embed(inputs)
        bigruout = self.bigru(em_out)
        masked_encoded = F.SequenceMask(bigruout,
                                        sequence_length=length,
                                        use_sequence_length=True).transpose((1,0,2))
        dense_out = self.dense_prev(masked_encoded)
        outs = self.dense(dense_out) 
        return(outs)

In [16]:
ctx = mx.gpu()

model = IntentClassification(vocab_size = len(vocab_sent.idx_to_token), 
                             vocab_out_size=len(vocab_intent.idx_to_token), num_embed=50, seq_len=seq_len, hidden_size=30)

In [17]:
model.initialize(mx.initializer.Xavier(), ctx=ctx)

In [18]:
trainer = gluon.Trainer(model.collect_params(),"Adam")
loss = gluon.loss.SoftmaxCELoss() 

In [19]:
model.hybridize()

In [21]:
def evaluate_accuracy(model, data_iter, ctx=ctx):
    acc = mx.metric.Accuracy()
    for i, (data, length, label) in enumerate(data_iter):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        length = length.as_in_context(ctx)
        output = model(data.T, length)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return(acc.get()[1])

In [22]:
def calculate_loss(model, data_iter, loss_obj, ctx=ctx):
    test_loss = []
    for i, (te_data, te_length, te_label) in enumerate(data_iter):
        te_data = te_data.as_in_context(ctx)
        te_label = te_label.as_in_context(ctx)
        te_length = te_length.as_in_context(ctx)
        te_output = model(te_data.T, te_length)
        loss_te = loss_obj(te_output, te_label)
        curr_loss = nd.mean(loss_te).asscalar()
        test_loss.append(curr_loss)
    return(np.mean(test_loss))

In [23]:
epochs = 100


tot_test_loss = []
tot_test_accu = []
tot_train_loss = []
tot_train_accu = []
tot_valid_accu = [] 
for e in range(epochs):
    #batch training 
    for i, (data, length, label) in enumerate(tqdm(train_dataloader)):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        length = length.as_in_context(ctx)
        with autograd.record():
            output = model(data.T, length)
            loss_ = loss(output, label)
            loss_.backward()
        trainer.step(data.shape[0])

    #caculate test loss
    if e % 10 == 0: 
        test_loss = calculate_loss(model, test_dataloader, loss_obj = loss, ctx=ctx) 
        train_loss = calculate_loss(model, train_dataloader, loss_obj = loss, ctx=ctx) 
        test_accu = evaluate_accuracy(model, test_dataloader,  ctx=ctx)
        train_accu = evaluate_accuracy(model, train_dataloader,  ctx=ctx)
        valid_accu = evaluate_accuracy(model, valid_dataloader,  ctx=ctx)

        print("Epoch %s. Train Loss: %s, Test Loss : %s," \
        " Test Accuracy : %s," \
        " Train Accuracy : %s : Valid Accuracy : %s" % (e, train_loss, test_loss, test_accu, train_accu, valid_accu))    
        tot_test_loss.append(test_loss)
        tot_train_loss.append(train_loss)
        tot_test_accu.append(test_accu)
        tot_train_accu.append(train_accu)
        tot_valid_accu.append(valid_accu)

 10%|█         | 27/270 [00:00<00:00, 264.76it/s]

Epoch 0. Train Loss: 0.16074908, Test Loss : 0.20620774, Test Accuracy : 0.9466666666666667, Train Accuracy : 0.9591358024691358 : Valid Accuracy : 0.959


 11%|█         | 30/270 [00:00<00:00, 294.48it/s]

Epoch 10. Train Loss: 0.0014749003, Test Loss : 0.07183168, Test Accuracy : 0.9877777777777778, Train Accuracy : 0.9996296296296296 : Valid Accuracy : 0.994


 11%|█▏        | 31/270 [00:00<00:00, 309.35it/s]

...

 11%|█         | 30/270 [00:00<00:00, 296.61it/s]

Epoch 80. Train Loss: 1.1714884e-08, Test Loss : 0.1397898, Test Accuracy : 0.9888888888888889, Train Accuracy : 1.0 : Valid Accuracy : 0.992


 11%|█▏        | 31/270 [00:00<00:00, 307.84it/s]

Epoch 90. Train Loss: 3.1494796e-09, Test Loss : 0.14051336, Test Accuracy : 0.9888888888888889, Train Accuracy : 1.0 : Valid Accuracy : 0.993


100%|██████████| 270/270 [00:00<00:00, 298.53it/s]


#### Model export and Visualize 

In [55]:
model.export("model")

Netron으로 네트워크 시각화 

- https://lutzroeder.github.io/netron/
- 저장된 `model-symbol.json`을 입력해 시각화 

In [61]:
load_model = gluon.nn.SymbolBlock.imports("model-symbol.json", ['data0', 'data1'], "model-0000.params")

In [76]:
def get_intent(sent):
    sent_len = len(sent)
    coded_sent = vocab_sent[length_clip(list(sent))]
    co = nd.array(coded_sent).expand_dims(axis=1)
    ret_code = load_model(co, nd.array([sent_len,]))
    ret_seq = vocab_intent.to_tokens(ret_code.argmax(axis=1).asnumpy().astype('int').tolist())
    return(''.join(ret_seq))

### TODO
- 개별 Intent와 Entity 모형을 하나의 모형으로 구축해본다. (Multi-Task Learning) 
  - 분류 성능이 좋아지는가? 학습 수렴 속도는 어떠한가?